The intention of this notebook is to ***sample subsets of data*** based on their category from the  T1626 dataset provided by Merck&Co.

The cross-validation on T1626 with the models based on the *Merck&Co+* features was based on this dataset.

However, this was later redone in *../paper_datasets_T1626/* in a standartized and consistent way for compatibility with *T251* and *T96*

##Notebook Setup##

In [0]:
#Imports:
import os
import numpy as np
import pandas as pd

In [0]:
#Connect to google drive:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Set up file paths:
data_folder_loc = "gdrive/My Drive/iGEM/Databases/Merck&Co/Data/Source/"
merck_and_co_excel_loc = data_folder_loc + "merck_and_co.xls"
target_data_folder_loc = "gdrive/My Drive/iGEM/Databases/Merck&Co/Data/T1626/own_baseline_xgboost_T1626/"
print(os.path.isdir(data_folder_loc))
print(os.path.isdir(target_data_folder_loc))
print(os.path.isfile(merck_and_co_excel_loc))

True
True
True


## Sample Data ##

In [0]:
#Read excel table T1626:
T1626_table = pd.read_excel(io=merck_and_co_excel_loc, sheet_name=0, header=0)

In [0]:
#remove characters from column heads that could couse problems
T1626_table.columns = map(lambda x: x.replace("[", "").replace("]", "").replace(">", "_greater_").replace("<", "_smaller_").replace(" ", "_").replace(".", "_").replace("Δ", "delta_").replace("Å", "A").replace("(", "_").replace(")", "_"), T1626_table.columns)

### Define semantically grouped lists of features  ###


In [0]:
########################################
target_label = 'dtm'

sequence_identity_related_columns = ['Mutation', 'dtm', 'wt', 'resi', 'mut', 'pdbId', 'protein']

features_type_A = ['beta_first', 'beta_second', 'beta_diff', 'alpha_first', 'alpha_second', 'alpha_diff', 'coil_first', 'coil_second', 'coil_diff', 'mutability_first', 'mutability_second', 'mutability_diff', 'flexibility_first', 'flexibility_second', 'flexibility_diff', 'aromatic_first', 'aromatic_second', 'aromatic_diff', 'polarity_first', 'polarity_second', 'polarity_diff', 'pi_first', 'pi_second', 'pi_diff', 'consensus_hydrophobicity_first', 'consensus_hydrophobicity_second', 'consensus_hydrophobicity_diff', 'hydrophobicity_first', 'hydrophobicity_second', 'hydrophobicity_diff', 'bulkiness_first', 'bulkiness_second', 'bulkiness_diff', 'mol_wt_first', 'mol_wt_second', 'mol_wt_diff', 'vdw_vol_first', 'vdw_vol_second', 'vdw_vol_diff', 'branched_first', 'branched_second', 'branched_diff', 'logd_first', 'logd_second', 'logd_diff', 'charge_first', 'charge_second', 'charge_diff', 'polarity2_first', 'polarity2_second', 'polarity2_diff', 'dg_o_w_first', 'dg_o_w_second', 'dg_o_w_diff']
features_type_L = ['wtss', 'totalarea', 'fractionburied', 'hydrophobicratio', 'hydrophobicarea']
# version where wtss is excluded, because its not available in all sub datasets
features_type_L_available = ['totalarea', 'fractionburied', 'hydrophobicratio', 'hydrophobicarea'] 
features_type_G = ['dmobility', 'ddipole_moment', 'dasa_vdw', 'deccen', 'dasa_hph', 'dhelicity', 'dzdipole', 'dr_solv', 'dapp_charge', 'dsed_const', 'dpi_3d', 'dzquadrupole', 'dasa_hyd', 'dzeta', 'dhenry', 'dpi_seq', 'dnet_charge', 'dvolume', 'dcoeff_fric', 'dr_gyr', 'ddebye', 'dhyd_moment', 'dcoeff_280']

feature_variant_polarity = 'variant_polarity'
features_energy_BL = ['r_bioluminate_delta_Stability_Solv_SA', 'r_bioluminate_delta_Stability_SelfCont', 'r_bioluminate_delta_Stability_Packing', 'r_bioluminate_delta_Stability_Covalent', 'r_bioluminate_delta_Stability_Reference', 'r_bioluminate_delta_Stability_vdW', 'r_bioluminate_delta_Stability_Hbond', 'r_bioluminate_delta_Stability_Coulomb', 'r_bioluminate_delta_Stability_Solv_GB', 'r_bioluminate_delta_Stability_Lipo', 'r_bioluminate_delta_Stability']
feature_energy_ddG_BL = 'r_bioluminate_delta_Stability' #PRObABLY?
features_energy_CART = ['cart_ddg', 'cart_fa_dun_dev', 'cart_cart_bonded', 'cart_fa_intra_sol_xover4', 'cart_dslf_fa13', 'cart_fa_intra_elec', 'cart_fa_dun_semi', 'cart_hbond_sr_bb', 'cart_fa_sol', 'cart_lk_ball_bridge', 'cart_rama_prepro', 'cart_hbond_sc', 'cart_hbond_lr_bb', 'cart_lk_ball_bridge_uncpl', 'cart_hbond_bb_sc', 'cart_lk_ball_iso', 'cart_fa_intra_rep_xover4', 'cart_fa_atr', 'cart_fa_rep', 'cart_lk_ball', 'cart_fa_dun_rot', 'cart_omega', 'cart_ref', 'cart_hxl_tors', 'cart_fa_intra_atr_xover4', 'cart_p_aa_pp', 'cart_fa_elec']
feature_energy_ddG_CART = 'cart_ddg'
features_energy_DS = ['ds_vdw', 'ds_elec', 'ds_entropy', 'ds_ddg', 'dS']
feature_energy_ddG_DS = 'ds_ddg'
features_energy_MOE = ['dasa', 'dEff_vdw', 'dEff_elc', 'dEsg', 'dEself', 'dh_ema', 'dh_emd', 'da_acc', 'da_don', 'da_asa_p', 'db_rotN', 'da_nH', 'dgly', 'da_heavy']
#feature_energy_ddG_MOE = #DONT KNOW REALLY?
#PROBLEM WITH MONO: DOCUMENTATION AND DATASET DONT MATCH, HOWEVER, DATASET CONSISTENT IN IT SELF
features_energy_MONO = ['mono_ddg', 'mono_fa_atr', 'mono_fa_rep', 'mono_fa_sol', 'mono_fa_intra_rep', 'mono_pro_close', 'mono_fa_pair', 'mono_hbond_sr_bb', 'mono_hbond_lr_bb', 'mono_hbond_bb_sc', 'mono_hbond_sc', 'mono_dslf_ss_dst', 'mono_dslf_cs_ang', 'mono_dslf_ss_dih', 'mono_dslf_ca_dih', 'mono_rama', 'mono_omega', 'mono_fa_dun', 'mono_p_aa_pp', 'mono_ref']
feature_energy_ddG_MONO = 'mono_ddg'

featrures_ALG_CART_tiE = features_type_A + features_type_L + features_type_G + features_energy_CART
features_all_used_by_paper = features_type_A + features_type_L + features_type_G + features_energy_BL + features_energy_CART + features_energy_DS + features_energy_MOE + features_energy_MONO
features_ALG = features_type_A + features_type_L + features_type_G 
features_E = features_energy_BL + features_energy_CART + features_energy_DS + features_energy_MOE + features_energy_MONO

HM_features = ['proteinAbbreviation', 'proteinCommon', 'Organism', 'source', 'ddG', 'delta_Tmexp', 'Tmexp_wt', 'delta_delta_Hmexp', 'delta_Hmexp_wt', 'delta_delta_CPexp', 'delta_CPexp_wt', 'delta_delta_Gexp_T_', 'T', 'Nres', 'R__A_', 'Ref_', 'pH', 'Exp_Tech_', 'wtPolarity', 'mutPolarity']
HM_feature_sequemce_Tm = 'Tmexp_wt'
HM_feature_organism_growth_T = 'T'
HM_feature_organism_pH = 'pH'
HM_feature_sequence_len = 'Nres'
HM_features_promising = [HM_feature_sequemce_Tm,HM_feature_organism_growth_T,HM_feature_organism_pH,HM_feature_sequence_len]

#features that dont make sense to use beacuase they would probably not be available beforehand:
HM_unrealistic_potentially_useful_features = ['ddG','delta_delta_Hmexp', 'delta_delta_CPexp']
#features that I dont really know what they mean
HM_potentially_useful_features = ['delta_Hmexp_wt', 'delta_CPexp_wt']

features_all_available_that_make_sense = features_all_used_by_paper + HM_features_promising

### Get overview over dataframe ###

In [0]:
str(list(T1626_table.columns))

"['Mutation', 'dtm', 'wt', 'resi', 'mut', 'pdbId', 'protein', 'beta_first', 'beta_second', 'beta_diff', 'alpha_first', 'alpha_second', 'alpha_diff', 'coil_first', 'coil_second', 'coil_diff', 'mutability_first', 'mutability_second', 'mutability_diff', 'flexibility_first', 'flexibility_second', 'flexibility_diff', 'aromatic_first', 'aromatic_second', 'aromatic_diff', 'polarity_first', 'polarity_second', 'polarity_diff', 'pi_first', 'pi_second', 'pi_diff', 'consensus_hydrophobicity_first', 'consensus_hydrophobicity_second', 'consensus_hydrophobicity_diff', 'hydrophobicity_first', 'hydrophobicity_second', 'hydrophobicity_diff', 'bulkiness_first', 'bulkiness_second', 'bulkiness_diff', 'mol_wt_first', 'mol_wt_second', 'mol_wt_diff', 'vdw_vol_first', 'vdw_vol_second', 'vdw_vol_diff', 'branched_first', 'branched_second', 'branched_diff', 'logd_first', 'logd_second', 'logd_diff', 'charge_first', 'charge_second', 'charge_diff', 'polarity2_first', 'polarity2_second', 'polarity2_diff', 'dg_o_w_fir

In [0]:
(T1626_table.dtm - T1626_table.delta_Tmexp).sum() #should be 0 I think,neglegible though, might be due to roundding error?

-0.020000000000000018

In [0]:
T1626_table.head()

,Mutation,dtm,wt,resi,mut,pdbId,protein,beta_first,beta_second,beta_diff,...,delta_CPexp_wt,delta_delta_Gexp_T_,T,Nres,R__A_,Ref_,pH,Exp_Tech_,wtPolarity,mutPolarity
0,1AKY@A@I213F,-7.7,I,213,F,1AKY,Adenylate Kinase,1.71,1.43,-0.28,...,-8.5,8.0,25.0,220.0,1.63,1.0,7.5,FL,apolar,apolar
1,1AKY@A@N169D,-0.6,N,169,D,1AKY,Adenylate Kinase,0.63,0.55,-0.08,...,-8.5,9.0,25.0,220.0,1.63,1.0,7.5,FL,polar,charged
2,1AKY@A@Q48E,-1.3,Q,48,E,1AKY,Adenylate Kinase,0.76,0.72,-0.04,...,-8.5,4.0,25.0,220.0,1.63,1.0,7.7,FL,polar,charged
3,1AKY@A@T110H,-4.8,T,110,H,1AKY,Adenylate Kinase,1.23,0.99,-0.24,...,-8.5,11.0,25.0,220.0,1.63,1.0,7.6,FL,polar,charged
4,1AKY@A@T77H,-1.1,T,77,H,1AKY,Adenylate Kinase,1.23,0.99,-0.24,...,-8.5,9.0,25.0,220.0,1.63,1.0,7.5,FL,polar,charged


In [0]:
T1626_table.describe()

,dtm,resi,beta_first,beta_second,beta_diff,alpha_first,alpha_second,alpha_diff,coil_first,coil_second,...,Tmexp_wt,delta_delta_Hmexp,delta_Hmexp_wt,delta_delta_CPexp,delta_CPexp_wt,T,Nres,R__A_,Ref_,pH
count,1626.000000,1626.000000,1626.000000,1626.000000,1626.000000,1626.000000,1626.000000,1626.000000,1626.000000,1626.000000,...,1572.000000,939.000000,1263.000000,246.000000,1067.000000,1089.000000,1572.000000,1572.000000,1555.000000,1571.000000
mean,-2.704071,93.108856,1.069563,1.023419,-0.046144,0.981464,1.051298,0.069834,0.978721,0.943684,...,62.589059,30.510011,-443.660333,0.323171,-8.270384,40.903581,175.622137,1.638308,1559.218006,6.048759
std,5.310652,102.734076,0.437156,0.388831,0.526734,0.250148,0.280576,0.381110,0.310684,0.292165,...,12.040779,59.913490,138.941126,1.965621,3.613720,19.320414,113.722376,0.388625,15580.848139,1.993024
min,-19.900000,1.000000,0.440000,0.440000,-1.420000,0.470000,0.470000,-0.920000,0.590000,0.590000,...,34.800000,-238.500000,-907.900000,-9.600000,-20.100000,12.000000,51.000000,0.750000,1.000000,2.000000
25%,-5.400000,42.000000,0.720000,0.750000,-0.350000,0.820000,0.820000,-0.150000,0.760000,0.760000,...,53.100000,0.000000,-548.100000,-0.475000,-10.500000,25.000000,124.000000,1.480000,52.000000,5.000000
50%,-1.700000,76.000000,0.910000,0.850000,-0.010000,0.920000,1.040000,0.070000,0.890000,0.800000,...,63.350000,20.900000,-468.600000,0.100000,-7.500000,25.000000,149.000000,1.630000,108.000000,7.000000
75%,0.300000,117.000000,1.430000,1.300000,0.250000,1.170000,1.350000,0.337500,1.240000,1.070000,...,65.100000,57.750000,-337.200000,1.000000,-6.500000,56.000000,164.000000,1.900000,205.000000,7.400000
max,16.600000,1833.000000,1.860000,1.860000,1.420000,1.390000,1.390000,0.920000,1.720000,1.720000,...,138.800000,440.600000,-126.800000,10.500000,-1.300000,86.000000,628.000000,2.500000,219220.000000,11.000000


In [0]:
T1626_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1626 entries, 0 to 1625
Columns: 187 entries, Mutation to mutPolarity
dtypes: float64(145), int64(28), object(14)
memory usage: 2.3+ MB


### Create data for XGBoost ###

In [0]:
#create target label 
T1626_table[target_label].to_pickle(target_data_folder_loc+"y_label_own_baseline_xgboost_T1626.pkl")
T1626_table[target_label].to_csv(target_data_folder_loc+"y_label_own_baseline_xgboost_T1626.csv")

In [0]:
#create paper features array
T1626_table[features_all_used_by_paper].to_pickle(target_data_folder_loc+"X_paper_own_baseline_xgboost_T1626.pkl")
T1626_table[features_all_used_by_paper].to_csv(target_data_folder_loc+"X_paper_own_baseline_xgboost_T1626.csv")

In [0]:
#create paper_ext features array
T1626_table[features_all_available_that_make_sense].to_pickle(target_data_folder_loc+"X_paper_ext_own_baseline_xgboost_T1626.pkl")
T1626_table[features_all_available_that_make_sense].to_csv(target_data_folder_loc+"X_paper_ext_own_baseline_xgboost_T1626.csv")

In [0]:
#create ALG_cart_E
T1626_table[featrures_ALG_CART_tiE].to_pickle(target_data_folder_loc+"X_paper_ALG_CART_tiE_own_baseline_xgboost_T1626.pkl")
T1626_table[featrures_ALG_CART_tiE].to_csv(target_data_folder_loc+"X_paper_ALG_CART_tiE_own_baseline_xgboost_T1626.csv")

In [0]:
#create ALG
T1626_table[features_ALG].to_pickle(target_data_folder_loc+"X_paper_ALG_own_baseline_xgboost_T1626.pkl")
T1626_table[features_ALG].to_csv(target_data_folder_loc+"X_paper_ALG_own_baseline_xgboost_T1626.csv")

In [0]:
#create E
T1626_table[features_E].to_pickle(target_data_folder_loc+"X_paper_E_own_baseline_xgboost_T1626.pkl")
T1626_table[features_E].to_csv(target_data_folder_loc+"X_paper_E_own_baseline_xgboost_T1626.csv")